# 如何将运行时值传递给工具
import Prerequisites from "@theme/Prerequisites";import Compatibility from "@theme/Compatibility";
<Prerequisites titlesAndLinks={[
["聊天模型", "/docs/concepts/chat_models"]["LangChain 工具", "/docs/concepts/tools"]["如何创建工具", "/docs/how_to/custom_tools"]["如何使用模型调用工具", "/docs/how_to/tool_calling"]]} />

<Compatibility packagesAndVersions={[
["langchain-core", "0.2.21"]]} />
你可能需要将仅在运行时才知晓的值绑定到某个[工具](/docs/concepts/tools/)。例如，该工具的逻辑可能需要使用发起请求的用户的ID。
大多数情况下，此类值不应由大语言模型控制。事实上，允许大语言模型操作用户ID可能会导致安全隐患。
相反，LLM 只应控制那些本应由 LLM 控制的工具参数，而其他参数（如用户 ID）应由应用程序逻辑固定。
本操作指南将向您展示如何防止模型在运行时生成特定工具参数并直接注入它们。
:::info 与LangGraph配合使用
如果您正在使用LangGraph，请参考[此操作指南](https://langchain-ai.github.io/langgraph/how-tos/pass-run-time-values-to-tools/)以下展示了如何创建一个代理，用于跟踪指定用户最喜欢的宠物。好的，请提供需要翻译的英文内容，我会将其翻译成中文并保持原有的Markdown格式。

我们可以将它们绑定到聊天模型如下：
import ChatModelTabs from "@theme/ChatModelTabs";
<ChatModelTabs
customVarName="llm"```markdown
overrideParams={{fireworks: {model: "accounts/fireworks/models/firefunction-v1", kwargs: "temperature=0"}}}
```/>

In [1]:
# | output: false
# | echo: false

import os
from getpass import getpass

from langchain_openai import ChatOpenAI

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## 对模型隐藏参数
我们可以使用 `InjectedToolArg` 注解来标记工具（Tool）的某些参数（例如 `user_id`），这些参数将在运行时被注入，意味着它们不应由模型生成。

In [2]:
from typing import List

from langchain_core.tools import InjectedToolArg, tool
from typing_extensions import Annotated

user_to_pets = {}


@tool(parse_docstring=True)
def update_favorite_pets(
    pets: List[str], user_id: Annotated[str, InjectedToolArg]
) -> None:
    """Add the list of favorite pets.

    Args:
        pets: List of favorite pets to set.
        user_id: User's ID.
    """
    user_to_pets[user_id] = pets


@tool(parse_docstring=True)
def delete_favorite_pets(user_id: Annotated[str, InjectedToolArg]) -> None:
    """Delete the list of favorite pets.

    Args:
        user_id: User's ID.
    """
    if user_id in user_to_pets:
        del user_to_pets[user_id]


@tool(parse_docstring=True)
def list_favorite_pets(user_id: Annotated[str, InjectedToolArg]) -> None:
    """List favorite pets if any.

    Args:
        user_id: User's ID.
    """
    return user_to_pets.get(user_id, [])

如果我们查看这些工具的输入模式，会发现`user_id`仍然被列出：

In [3]:
update_favorite_pets.get_input_schema().schema()

{'description': 'Add the list of favorite pets.',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'},
  'user_id': {'description': "User's ID.",
   'title': 'User Id',
   'type': 'string'}},
 'required': ['pets', 'user_id'],
 'title': 'update_favorite_petsSchema',
 'type': 'object'}

但如果我们查看工具调用模式（即传递给模型以进行工具调用的数据结构），会发现 `user_id` 字段已被移除：

In [4]:
update_favorite_pets.tool_call_schema.schema()

{'description': 'Add the list of favorite pets.',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'}},
 'required': ['pets'],
 'title': 'update_favorite_pets',
 'type': 'object'}

因此，当我们调用工具时，需要传入 `user_id`：

In [5]:
user_id = "123"
update_favorite_pets.invoke({"pets": ["lizard", "dog"], "user_id": user_id})
print(user_to_pets)
print(list_favorite_pets.invoke({"user_id": user_id}))

{'123': ['lizard', 'dog']}
['lizard', 'dog']


但当模型调用工具时，不会生成 `user_id` 参数：

In [6]:
tools = [
    update_favorite_pets,
    delete_favorite_pets,
    list_favorite_pets,
]
llm_with_tools = llm.bind_tools(tools)
ai_msg = llm_with_tools.invoke("my favorite animals are cats and parrots")
ai_msg.tool_calls

[{'name': 'update_favorite_pets',
  'args': {'pets': ['cats', 'parrots']},
  'id': 'call_pZ6XVREGh1L0BBSsiGIf1xVm',
  'type': 'tool_call'}]

## 在运行时注入参数

如果我们要实际执行模型生成的工具调用来使用这些工具，就需要自行注入 user_id：

In [7]:
from copy import deepcopy

from langchain_core.runnables import chain


@chain
def inject_user_id(ai_msg):
    tool_calls = []
    for tool_call in ai_msg.tool_calls:
        tool_call_copy = deepcopy(tool_call)
        tool_call_copy["args"]["user_id"] = user_id
        tool_calls.append(tool_call_copy)
    return tool_calls


inject_user_id.invoke(ai_msg)

[{'name': 'update_favorite_pets',
  'args': {'pets': ['cats', 'parrots'], 'user_id': '123'},
  'id': 'call_pZ6XVREGh1L0BBSsiGIf1xVm',
  'type': 'tool_call'}]

现在，我们可以将模型、注入代码和实际工具串联起来，创建一个执行工具的链式流程：

In [8]:
tool_map = {tool.name: tool for tool in tools}


@chain
def tool_router(tool_call):
    return tool_map[tool_call["name"]]


chain = llm_with_tools | inject_user_id | tool_router.map()
chain.invoke("my favorite animals are cats and parrots")

[ToolMessage(content='null', name='update_favorite_pets', tool_call_id='call_oYCD0THSedHTbwNAY3NW6uUj')]

查看 user_to_pets 字典，可以看到它已经更新为包含猫和鹦鹉：

In [9]:
user_to_pets

{'123': ['cats', 'parrots']}

## 其他参数标注方式
以下是几种其他标注工具参数的方法：

In [10]:
from langchain_core.tools import BaseTool
from pydantic import BaseModel, Field


class UpdateFavoritePetsSchema(BaseModel):
    """Update list of favorite pets"""

    pets: List[str] = Field(..., description="List of favorite pets to set.")
    user_id: Annotated[str, InjectedToolArg] = Field(..., description="User's ID.")


@tool(args_schema=UpdateFavoritePetsSchema)
def update_favorite_pets(pets, user_id):
    user_to_pets[user_id] = pets


update_favorite_pets.get_input_schema().schema()

{'description': 'Update list of favorite pets',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'},
  'user_id': {'description': "User's ID.",
   'title': 'User Id',
   'type': 'string'}},
 'required': ['pets', 'user_id'],
 'title': 'UpdateFavoritePetsSchema',
 'type': 'object'}

In [11]:
update_favorite_pets.tool_call_schema.schema()

{'description': 'Update list of favorite pets',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'}},
 'required': ['pets'],
 'title': 'update_favorite_pets',
 'type': 'object'}

In [12]:
from typing import Optional, Type


class UpdateFavoritePets(BaseTool):
    name: str = "update_favorite_pets"
    description: str = "Update list of favorite pets"
    args_schema: Optional[Type[BaseModel]] = UpdateFavoritePetsSchema

    def _run(self, pets, user_id):
        user_to_pets[user_id] = pets


UpdateFavoritePets().get_input_schema().schema()

{'description': 'Update list of favorite pets',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'},
  'user_id': {'description': "User's ID.",
   'title': 'User Id',
   'type': 'string'}},
 'required': ['pets', 'user_id'],
 'title': 'UpdateFavoritePetsSchema',
 'type': 'object'}

In [13]:
UpdateFavoritePets().tool_call_schema.schema()

{'description': 'Update list of favorite pets',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'}},
 'required': ['pets'],
 'title': 'update_favorite_pets',
 'type': 'object'}

In [14]:
class UpdateFavoritePets2(BaseTool):
    name: str = "update_favorite_pets"
    description: str = "Update list of favorite pets"

    def _run(self, pets: List[str], user_id: Annotated[str, InjectedToolArg]) -> None:
        user_to_pets[user_id] = pets


UpdateFavoritePets2().get_input_schema().schema()

{'description': 'Use the tool.\n\nAdd run_manager: Optional[CallbackManagerForToolRun] = None\nto child implementations to enable tracing.',
 'properties': {'pets': {'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'},
  'user_id': {'title': 'User Id', 'type': 'string'}},
 'required': ['pets', 'user_id'],
 'title': 'update_favorite_petsSchema',
 'type': 'object'}

In [15]:
UpdateFavoritePets2().tool_call_schema.schema()

{'description': 'Update list of favorite pets',
 'properties': {'pets': {'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'}},
 'required': ['pets'],
 'title': 'update_favorite_pets',
 'type': 'object'}